<a href="https://colab.research.google.com/github/ruhullahil/gcolab/blob/master/CNN_hard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

In [0]:
class Convolution:
    '''
    Applies Convolution to the input image
    - image is a numpy 2d array
    
    filter_size is a tuple which takes the row and column size of 2d ConvLayer
    num_filters is the number of filters/kernel to use
    
    This convolutional layer uses valid padding
    '''
    def __init__(self, filter_size, num_filters, scale_value=9):
        self.num_filters = num_filters
        self.n, self.m = filter_size
        print(self.n,self.m)
        
        # filters is a 3d array with dimensions (num_filters, n, m)
        # We divide by 9 to reduce the variance of our initial values
        self.filters = np.random.randn(num_filters, self.n, self.m) 
        print("Kernel shape ", self.filters.shape)
        
    def iterate_regions(self, image):
        '''
        Generates all possible nxm image regions using valid padding.
        - image is a 2d numpy array
        '''
        h, w = image.shape
        for i in range(h-(self.n-1)):
            for j in range(w-(self.m-1)):
                img_region = image[i:i+self.n, j:j+self.m]
                yield img_region, i, j
    
    def forward(self, input):
        '''
        Performs a forward pass of the conv layer using the given input.
        Returns a 3d numpy array with dimensions (h, w, num_filters).
        - input is a 2d numpy array
        '''
        h, w = input.shape
        output = np.zeros((h-(self.n-1), w-(self.m-1), self.num_filters))
        
        for img_region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(img_region * self.filters, axis=(1, 2))
        return output

In [0]:
class MaxPool:
    '''
    Generates non-overlapping nxm image regions to pool over.
    - image is a 2d numpy array
    
    poolShape is the row and column size of the pool
    '''
    def __init__(self, poolShape):
        self.n, self.m = poolShape
    
    def iterate_regions(self, image):
        h, w = image.shape[:2]
        
        new_h = h // self.n
        new_w = w // self.m
        
        for i in range(new_h):
            for j in range(new_w):
                p_start = i*self.n
                p_end = p_start+self.n
                q_start = i*self.m
                q_end = q_start+self.m
                im_region = image[p_start:p_end, q_start:q_end]
                yield im_region, i, j
    
    def forward(self, input):
        '''
        Performs a forward pass of the maxpool layer using the given input.
        Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
        - input is a 3d numpy array with dimensions (h, w, num_filters)
        '''
        h, w, num_filters = input.shape
        output = np.zeros((h // self.n, w // self.m, num_filters))
        
        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))
        return output

In [0]:
class Softmax:
    '''
    Input parameters:
    input_len = total nodes
    nodes = total number of classes, refers to the nodes of softmax
        
    Flatterns the whole input dimention and applies softmax to the flattened vector
    '''
    def __init__(self, input_len, softmax_nodes):
        # We divide by input_len to reduce the variance of our initial values
        self.weights = np.random.randn(input_len, softmax_nodes) / input_len
        self.biases = np.zeros(softmax_nodes)
        
    def forward(self, input):
        '''
        Performs a forward pass of the softmax layer using the given input.
        Returns a 1d numpy array containing the respective probability values.
        - input can be any array with any dimensions.
        '''
        input = input.flatten()
        input_len, softmax_nodes = self.weights.shape
        
        totals = np.dot(input, self.weights) + self.biases
        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

In [0]:
def feedToCNN(image, label, conv, pool, softmax):
    '''
    Completes a forward pass of the CNN and calculates the accuracy and
    cross-entropy loss.
    - image is a 2d numpy array
    - label is a digit
    - conv is Convolution object
    - pool is Maxpool object
    - softmax is Softmax object
    '''
    
    # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
    # to work with. This is standard practice.
    out = conv.forward((image / 255) - 0.5)
    print(np.array(out).shape)
    out = pool.forward(out)
    out = softmax.forward(out)
    
    # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0
    
    return out, loss, acc

In [0]:
def plotFigs(images, title=None):
    w=10
    h=10
    
    fig=plt.figure(figsize=(8, 8))
    if title:
        fig.suptitle(title)
 
    rows = images.shape[-1]//2
    columns = rows + (images.shape[-1]%2)
    
    for i in range(images.shape[-1]):
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(images[:, :, i], cmap='gray')

    plt.show()

In [0]:
# Data of mnist digit
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Defining Layers of CNN
conv = Convolution((3, 3), 8)
pool = MaxPool((2, 2))
softmax = Softmax(13 * 13 * 8, 10)

loss = 0
num_correct = 0
for i, (im, label) in enumerate(zip(x_train[:1000], y_train[:1000])):
  _, l, acc = feedToCNN(im, label, conv, pool, softmax)
  loss += l
  num_correct += acc

  # Print stats every 100 steps.
  if i % 100 == 99:
    print(
      '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%'
      %(i + 1, loss / 100, num_correct)
    )
    loss, num_correct = (0, 0)

3 3
Kernel shape  (8, 3, 3)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
(26, 26, 8)
